In [252]:
# Install and run this if you haven't already
# !pip install matplotlib  
# !pip install scipy

import os
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

files_to_delete = [
    "verbatim_questions_only.csv",
    "verbatim_A.csv",
    "verbatim_B.csv",
    "verbatim_C.csv",
    "verbatim_D.csv",
    "value_A.csv",
    "value_B.csv",
    "value_C.csv",
    "value_D.csv",
    "rrnl_all_tidy_cleaned_noUndefined.csv"
]

for file in files_to_delete:
    if os.path.exists(file):
        os.remove(file)
        print(f"Deleted: {file}")
    else:
        print(f"Not found: {file}")   

Deleted: verbatim_questions_only.csv
Deleted: verbatim_A.csv
Deleted: verbatim_B.csv
Deleted: verbatim_C.csv
Deleted: verbatim_D.csv
Deleted: value_A.csv
Deleted: value_B.csv
Deleted: value_C.csv
Deleted: value_D.csv
Deleted: rrnl_all_tidy_cleaned_noUndefined.csv


In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


In [ ]:
df = pd.read_csv("rrnl_all_tidy(6).csv")

original_rows = len(df)
print("Number of rows:", original_rows)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
columns_to_drop = [
    "Reject reason",
    "reject time",
    "instruction",
    "description",
    "trialNumber"
]

df = df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
df.columns

In [ ]:
original_rows = len(df)

df = df[df["correctAnswer"].str.strip().str.lower() != "undefined"]

new_rows = len(df)

print("Rows removed:", original_rows - new_rows)
print("Remaining rows:", new_rows)


In [ ]:
df.to_csv("rrnl_all_tidy_cleaned_noUndefined.csv", index=False)

### Verbatim Questions Code

In [ ]:
# Verbatim Qs
verbatim_questions = [
    "Is fasting blood glucose normal?",
    "Is two hours postprandial blood glucose normal?",
    "Is systolic normal?",
    "Is diastolic normal?"
]


In [ ]:
# Grab only rows with verbatim questions
verbatim_df = df[df["responsePrompt"].isin(verbatim_questions)].copy()

verbatim_df.head()


In [ ]:
#Verbatim CSV 
verbatim_df.to_csv("verbatim_questions_only.csv", index=False)


In [ ]:
verbatim_df["correct"] = (
    verbatim_df["answer"].astype(str).str.strip().str.lower() ==
    verbatim_df["correctAnswer"].astype(str).str.strip().str.lower()
).astype(int)

In [ ]:
verbatim_df["graphGroup"] = verbatim_df["trialId"].str[0]

In [ ]:
accuracy_by_graph = (
    verbatim_df
    .groupby("graphGroup")["correct"]
    .mean()
    .mul(100)
    .round(2)
)

print(accuracy_by_graph)

accuracy_std = verbatim_df.groupby("graphGroup")["correct"].std()

group_counts = verbatim_df.groupby("graphGroup")["correct"].count()

accuracy_se = (accuracy_std / np.sqrt(group_counts)) * 100

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(
    ['V-A', 'V-B', 'V-C', 'V-D'],   
    accuracy_by_graph.values,          
    yerr=accuracy_std.values,          
    fmt='o',                          
    linestyle='--',                   
    linewidth=2, 
    markersize=8, 
    capsize=5,                            
    label='Participants ages 18-40'
)
plt.xlabel('Graph Group')
plt.ylabel('Accuracy rate (%)')
plt.title('Verbatim comprehension task')
plt.grid(True, alpha=0.3)
plt.ylim([95, 100])
plt.legend()
plt.show()


In [ ]:
A_df = verbatim_df[verbatim_df["graphGroup"] == "A"].copy()
B_df = verbatim_df[verbatim_df["graphGroup"] == "B"].copy()
C_df = verbatim_df[verbatim_df["graphGroup"] == "C"].copy()
D_df = verbatim_df[verbatim_df["graphGroup"] == "D"].copy()


In [ ]:
A_df.to_csv("verbatim_A.csv", index=False)
B_df.to_csv("verbatim_B.csv", index=False)
C_df.to_csv("verbatim_C.csv", index=False)
D_df.to_csv("verbatim_D.csv", index=False)

### Value Questions Code

In [ ]:
value_questions = [
    "What is the fasting blood glucose value?",
    "What is the two hours postprandial blood glucose value?",
    "What is the systolic value?",
    "What is the diastolic value?"
]

In [ ]:
value_df = df[df["responsePrompt"].isin(value_questions)].copy()

value_df.head()

In [ ]:
value_df["correct"] = (
    value_df["answer"].astype(str).str.strip().str.lower() ==
    value_df["correctAnswer"].astype(str).str.strip().str.lower()
).astype(int)

In [ ]:
value_df["graphGroup"] = value_df["trialId"].str[0]

In [ ]:
for group in ["A", "B", "C", "D"]:
    temp_df = value_df[value_df["graphGroup"] == group]
    temp_df.to_csv(f"value_{group}.csv", index=False)

In [ ]:
value_accuracy_by_graph = (
    value_df
    .groupby("graphGroup")["correct"]
    .mean()
    .mul(100)
    .round(2)
)

print(value_accuracy_by_graph)

value_accuracy_std = value_df.groupby("graphGroup")["correct"].std()

value_group_counts = value_df.groupby("graphGroup")["correct"].count()

value_accuracy_se = (value_accuracy_std / np.sqrt(group_counts)) * 100

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(
    ['V-A', 'V-B', 'V-C', 'V-D'],        
    value_accuracy_by_graph.values,        
    yerr=value_accuracy_std.values,            
    fmt='o',                               
    linestyle='--',                       
    linewidth=2, 
    markersize=8, 
    capsize=5,                             
    label='Participants ages 18-40'
)
plt.xlabel('Graph Group')
plt.ylabel('Accuracy rate (%)')
plt.title('Value interpretation task')
plt.grid(True, alpha=0.3)
plt.ylim([95, 100])
plt.legend()
plt.show()


### Table 1 STATS Calculations

In [ ]:
# VERBATIM ACCURACY STATS 

# Mean accuracy (%)
verbatim_mean = (
    verbatim_df
    .groupby("graphGroup")["correct"]
    .mean()
    .mul(100)
    .round(2)
)

# Standard deviation ( % scale)
verbatim_std = (
    verbatim_df
    .groupby("graphGroup")["correct"]
    .std()
    .mul(100)
    .round(2)
)

print("Verbatim Mean Accuracy (%) by Graph:")
print(verbatim_mean)
print("\nVerbatim Standard Deviation (%) by Graph:")
print(verbatim_std)

# ANOVA 
groups = [
    verbatim_df[verbatim_df["graphGroup"] == g]["correct"]
    for g in ["A", "B", "C", "D"]
]

F_value, p_value = stats.f_oneway(*groups)

print("\nVerbatim ANOVA Results:")
print("F-value:", round(F_value, 4))
print("p-value:", round(p_value, 6))

In [ ]:
# OVERALL VERBATIM

verbatim_overall_mean = verbatim_df["correct"].mean() * 100
verbatim_overall_sd = verbatim_df["correct"].std() * 100

print("\nOverall Verbatim Accuracy (All Graphs Combined):")
print("Mean (%):", round(verbatim_overall_mean, 2))
print("SD (%):", round(verbatim_overall_sd, 2))

In [ ]:
# VALUE ACCURACY STATS 

# Mean accuracy (%)
value_mean = (
    value_df
    .groupby("graphGroup")["correct"]
    .mean()
    .mul(100)
    .round(2)
)

# Standard deviation 
value_std = (
    value_df
    .groupby("graphGroup")["correct"]
    .std()
    .mul(100)
    .round(2)
)

print("Value Mean Accuracy (%) by Graph:")
print(value_mean)
print("\nValue Standard Deviation (%) by Graph:")
print(value_std)

#  ANOVA
value_groups = [
    value_df[value_df["graphGroup"] == g]["correct"]
    for g in ["A", "B", "C", "D"]
]

F_value_val, p_value_val = stats.f_oneway(*value_groups)

print("\nValue ANOVA Results:")
print("F-value:", round(F_value_val, 4))
print("p-value:", round(p_value_val, 6))

In [ ]:
# OVERALL VALUE

value_overall_mean = value_df["correct"].mean() * 100
value_overall_sd = value_df["correct"].std() * 100

print("\nOverall Value Accuracy (All Graphs Combined):")
print("Mean (%):", round(value_overall_mean, 2))
print("SD (%):", round(value_overall_sd, 2))